## ⛏️ DexRay - Enbedding

Notebook to genreate feature vectors the images using the DexRay pretrained model (removing the last layer).

In [ ]:
# IMPORT
from   tqdm       import tqdm
import pandas     as pd
import tensorflow as tf
import os

import Dexray

In [ ]:
# Initialize TQDM library for Pandas
tqdm.pandas()

In [ ]:
print("⚡ START ⚡")

### Parameters

In [ ]:
# Ground-Truth Dataset
INPUT_PATH  = "../../../../0_Data/CSV/0_AndroCatSet.csv"

# Output Path
OUTPUT_PATH = "../TMP/3b_DexrayData.csv"

TMP_PATH = "../TMP"
if not os.path.exists(TMP_PATH):
    os.makedirs(TMP_PATH)
    print("📁🆕 Folder created       :", TMP_PATH)
else:
    print("📁✅ Folder already exists:", TMP_PATH)

### 1. Load Data

In [ ]:
appsDF = pd.read_csv(INPUT_PATH,index_col=False)
print("#️⃣   APPS: {}".format(appsDF.shape[0]))

appsDF = appsDF.loc[:,['sha256','classID']]

### 2. Embedding

Load the pretrained model and remove the last layer.

In [ ]:
# Load the Pretrained Model
MODEL_PATH = "./dexrayModel"
loadedModel = tf.keras.models.load_model(MODEL_PATH, compile=False)
#loadedModel.summary()

# Remove the last layer
model = tf.keras.models.Sequential()
for layer in loadedModel.layers[:-1]:  # Exclude the last layer you want to remove
    model.add(layer)

# model.summary()

# Print the input shape
#print('Input shape:', model.layers[0] .input_shape)

### 3. Load Images and trasform to Tensors

In [ ]:
# Path Images
IMAGES_PATH = "../TMP/DexrayImages/"

print("⛏️ Loading the DexRay images as TF Tensors")
appsDF['imgFeatures'] = appsDF.progress_apply(lambda row: Dexray.getImgFeatures(row['sha256'], IMAGES_PATH), axis=1)

Embed the Images using the pretrained model

In [ ]:
print("⛏️ Encode Images using the pretrained DexRay Model")
appsDF['features'] = appsDF['imgFeatures'].progress_apply(Dexray.encodeImg,args={model})

# Filter
appsDF = appsDF.loc[:,['sha256','classID','features']]

In [ ]:
print("⛏️ Reorganizing features as lists")
appsDF['features'] = appsDF['features'].progress_apply(lambda x: x.tolist()[0])

### 4. Save everything to CSV

In [ ]:
# Save the result
appsDF.to_csv(OUTPUT_PATH, index=False)
appsDF.head(5)

In [ ]:
print("\n🔚 END \n")